In [ ]:
!pip install apache-flink

In [1]:
# Import necessary libraries from pyflink.table package
from pyflink.table import EnvironmentSettings, TableEnvironment

# Initialize the environment settings in streaming mode
env_settings = EnvironmentSettings.in_streaming_mode()

# Create a streaming table environment using the settings
table_env = TableEnvironment.create(env_settings)
table_env.execute_sql("""
    CREATE TABLE random_source (
        id BIGINT, 
        data TINYINT 
    ) WITH (
        'connector' = 'datagen',
        'fields.id.kind'='sequence',
        'fields.id.start'='1',
        'fields.id.end'='3',
        'fields.data.kind'='sequence',
        'fields.data.start'='4',
        'fields.data.end'='6'
    )
""")

# Retrieve the "random_source" table from the table environment
table = table_env.from_path("random_source")

# Execute the table operation and display the results
table.execute().print()


In [1]:
from pyflink.table import TableEnvironment, EnvironmentSettings

# Create a TableEnvironment
env_settings = EnvironmentSettings.in_streaming_mode()
t_env = TableEnvironment.create(env_settings)

# Specify connector and format jars
t_env.get_config().get_configuration().set_string(
    "pipeline.jars",
    "file:///workspace/spark/jdbc/flink-sql-connector-kafka-3.1.0-1.18.jar;"
    "file:///workspace/spark/jdbc/flink-sql-connector-elasticsearch7-3.0.1-1.17.jar"
)

### kafka

In [2]:
source_ddl = """
    CREATE TABLE temp_table(
        birthdate VARCHAR,
        blood_group VARCHAR,
        job VARCHAR,
        name VARCHAR,
        sex VARCHAR,
        ssn VARCHAR,
        uuid VARCHAR
    ) WITH (
        'connector' = 'kafka',
        'topic' = 'temp',
        'properties.bootstrap.servers' = 'kafka1:19091,kafka1:19092,kafka1:19093',
        'properties.group.id' = 'G1',
        'scan.startup.mode' = 'latest-offset',
        'format' = 'json'
    )
"""

# Define sink table DDL
sink_ddl = """
    CREATE TABLE sink_table(
        birthdate VARCHAR,
        blood_group VARCHAR,
        job VARCHAR,
        name VARCHAR,
        sex VARCHAR,
        ssn VARCHAR,
        uuid VARCHAR
    ) WITH (        
        'connector' = 'elasticsearch-7',
        'index' = 'demo_kafka_flink_streaming',
        'hosts' = 'es:9200',
        'format' = 'json'
    )
"""

In [3]:
# Execute DDL statement to create the source table
t_env.execute_sql(source_ddl)

# Retrieve the source table
source_table = t_env.from_path('temp_table')

source_table.print_schema()

In [4]:
# Define a SQL query to select all columns from the source table
sql_query = "SELECT sex, COUNT(sex) FROM temp_table GROUP BY sex"
# sql_query = "SELECT * FROM temp_table"

# Execute the query and retrieve the result table
result_table = t_env.sql_query(sql_query)

# Print the result table to the console
result_table.execute().print()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

### ElasticSearch

In [3]:
# Execute DDL statements to create tables
t_env.execute_sql(source_ddl)
t_env.execute_sql(sink_ddl)

# Retrieve the source table
source_table = t_env.from_path('temp_table')

print("Source Table Schema:")
source_table.print_schema()

Source Table Schema:


In [4]:
# Retrieve the sink table
sink_table = t_env.from_path('sink_table')

print("Sink Table Schema:")
sink_table.print_schema()

# Insert the processed data into the sink table
source_table.execute_insert('sink_table').wait()

Sink Table Schema:


Py4JJavaError: An error occurred while calling o107.await.
: java.util.concurrent.ExecutionException: org.apache.flink.table.api.TableException: Failed to wait job finish
	at java.base/java.util.concurrent.CompletableFuture.reportGet(CompletableFuture.java:395)
	at java.base/java.util.concurrent.CompletableFuture.get(CompletableFuture.java:2005)
	at org.apache.flink.table.api.internal.TableResultImpl.awaitInternal(TableResultImpl.java:122)
	at org.apache.flink.table.api.internal.TableResultImpl.await(TableResultImpl.java:85)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at org.apache.flink.api.python.shaded.py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at org.apach

In [4]:
from pyflink.table import EnvironmentSettings, TableEnvironment
from faker import Faker
import os

In [5]:
# Initialize Faker
fake = Faker()

# Generate fake data and convert it into a PyFlink table with column names
data = [(fake.name(), fake.city(), fake.state()) for _ in range(10)]  # Generate 10 rows of fake data

# Define column names
column_names = ["name", "city", "state"]

In [6]:
# Create a batch TableEnvironment
env_settings = EnvironmentSettings.in_streaming_mode()
table_env = TableEnvironment.create(env_settings)
table_env.get_config().get_configuration().set_string(
    "pipeline.jars",
    "file:///workspace/spark/jdbc/flink-sql-connector-kafka-3.1.0-1.18.jar"
)

# Create a PyFlink table with column names
table = table_env.from_elements(data, schema=column_names)

# Print the table
result = table.execute()

result.print()

In [5]:
for i in result.collect():
    print(i)

In [ ]:
table_env.create_temporary_view('source_table', table)

table_env.execute_sql(f"SELECT * FROM source_table ").print()

In [ ]:
table.to_pandas()

In [2]:
from pyflink.datastream.connectors.kafka import KafkaSource, KafkaOffsetResetStrategy, KafkaOffsetsInitializer, KafkaSink
from pyflink.common.serialization import SimpleStringSchema
from pyflink.common.watermark_strategy import WatermarkStrategy
from pyflink.datastream.stream_execution_environment import StreamExecutionEnvironment
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment, TableDescriptor, Schema, DataTypes

from pyflink.datastream.connectors.kafka import KafkaRecordSerializationSchema
from pyflink.datastream.formats.json import JsonRowSerializationSchema
from pyflink.common.typeinfo import Types

In [7]:
source = KafkaSource \
    .builder() \
    .set_bootstrap_servers('kafka1:19091,kafka1:19092,kafka1:19093') \
    .set_topics('demo') \
    .set_value_only_deserializer(SimpleStringSchema()) \
    .set_starting_offsets(KafkaOffsetsInitializer.earliest()) \
    .build()

In [ ]:
type(source)

In [8]:
env = StreamExecutionEnvironment.get_execution_environment()
t_env = StreamTableEnvironment.create(stream_execution_environment=env)

In [9]:
sink = KafkaSink.builder() \
    .set_bootstrap_servers('kafka1:19091,kafka1:19092,kafka1:19093') \
    .set_record_serializer(
        KafkaRecordSerializationSchema.builder()
            .set_topic("temp")
            .set_value_serialization_schema(
                JsonRowSerializationSchema.builder()
                .with_type_info(Types.ROW([Types.LONG(), Types.STRING()]))
                .build())
            .build()
    ) \
    .build()

In [10]:
ds = env.from_source(source, WatermarkStrategy.no_watermarks(), "Kafka Source")
ds.sink_to(sink)

In [ ]:
table_env.create_temporary_view('ds', ds)

table_env.execute_sql(f"SELECT * FROM ds ").print()

In [ ]:
import os

from pyflink.common import Row
from pyflink.common.typeinfo import Types
from pyflink.datastream import StreamExecutionEnvironment, RuntimeExecutionMode, TimeCharacteristic
from pyflink.datastream.connectors.kafka import (
    KafkaSink,
    KafkaRecordSerializationSchema,
    DeliveryGuarantee,
)
from pyflink.datastream.formats.json import JsonRowSerializationSchema

In [ ]:
BOOTSTRAP_SERVERS = os.getenv("BOOTSTRAP_SERVERS", "kafka1:19091,kafka1:19092,kafka1:19093")

env = StreamExecutionEnvironment.get_execution_environment()
env.set_runtime_mode(RuntimeExecutionMode.STREAMING)
env.set_stream_time_characteristic(TimeCharacteristic.ProcessingTime)
CURRENT_DIR = os.getcwd()  # Get the current working directory
# table_env.get_config().get_configuration().set_string(
#     "pipeline.jars",
#     "file:///" + CURRENT_DIR + "/jdbc/flink-sql-connector-kafka-3.1.0-1.18.jar",
# )

jar_files = ["flink-sql-connector-kafka-3.1.0-1.18.jar"]
jar_paths = tuple([f"file://{os.path.join(CURRENT_DIR, 'jars', name)}" for name in jar_files])
print(jar_paths)
env.add_jars(*jar_paths)

In [ ]:
value_type_info = Types.ROW_NAMED(
    field_names=["data"],
    field_types=[Types.STRING()],
)

source_stream = env.from_collection(
    collection=[
        [
            (("user1", "gold"), ("user2", "gold"), ("user5", "gold")),
            (("user3", "gold"), ("user4", "gold"), ("user6", "gold")),
        ]
    ]
)


sink = (
    KafkaSink.builder()
    .set_bootstrap_servers(BOOTSTRAP_SERVERS)
    .set_record_serializer(
        KafkaRecordSerializationSchema.builder()
        .set_topic("demo")
        .set_value_serialization_schema(
            JsonRowSerializationSchema.builder().with_type_info(value_type_info).build()
        )
        .build()
    )
    .set_delivery_guarantee(DeliveryGuarantee.AT_LEAST_ONCE)
    .build()
)

# source_stream.map(lambda e: Row(data=str(e)), output_type=value_type_info).print()
source_stream.map(lambda e: Row(data=str(e)), output_type=value_type_info).sink_to(sink).name(
    "sink-demo"
).uid("sink-demo")

env.execute("Kafka sink example.")